In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [73]:
## p(y_i = 1|X_it) = sigmoid(α_0 + α_1*X_1it + α_2*X_2it + γ_1*age_i + γ_2*female_i + γ_3*income_i + γ_4*cpi_t)
def gen_s2_tbl(df_s1, c, output_dir):
    # true quantity
    df_pivot_t = df_s1.pivot_table(index=['CustomerID', 'Date'], columns='ProductID', values='OrderQuantity', fill_value=0).reset_index()
    df_pivot_t.columns = ['CustomerID', 'Date', 'p1', 'p2','p3','p4']
    # model predicted quantity
    df_pivot_m = df_s1.pivot_table(index=['CustomerID', 'Date'], columns='ProductID', values='pred_s1', fill_value=0).reset_index()
    df_pivot_m.columns = ['CustomerID', 'Date', 'p1_m', 'p2_m','p3_m','p4_m']
    df_merged = pd.merge(df_pivot_t, df_s1[['CustomerID', 'Date','Temperature','Rainfall',
                                        'CPI','Female','Age','Income']].drop_duplicates(), on=['CustomerID','Date'])
    df_merged = pd.merge(df_merged, df_pivot_m, on=['CustomerID','Date'])

    df_merged['logit_t'] = c['intcpt']+c['p1']*df_merged['p1']+c['p1']*df_merged['p2']+c['p2']*df_merged['p3']+c['p3']*df_merged['p1']+\
    c['p4']*df_merged['p4']+c['age']*df_merged['Age']+c['female']*df_merged['Female']+ c['income']*df_merged['Income']+\
    c['cpi']*df_merged['CPI']+c['temp']*df_merged['Temperature']+c['rain']*df_merged['Rainfall']
    df_merged['logit_t'] = df_merged['logit_t']+np.random.normal(0,1, len(df_merged))

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    df_merged['membr_choice']=np.where(df_merged['logit_t'].apply(sigmoid)>0.7, 1, 0)
    df_merged.to_csv(output_dir, index=False)
    return df_merged


In [78]:
c = {
    'intcpt': 0.1,
    'p1':0.01,
    'p2':0.01,
    'p3':0.05,
    'p4':0.02,
    'female': -0.3,
    'age': 0.005,
    'income': 0.00001,
    'temp': 0.01,
    'rain': -0.04,
    'cpi': -0.01,
}
df_s1 = pd.read_csv('./data/stage1_model_data.csv')
df_s2 = gen_s2_tbl(df_s1=df_s1, c=c, output_dir='./data/stage2_data.csv')

In [79]:
df_s2['membr_choice'].sum()

10458

In [80]:
df_s2.head(2)

,CustomerID,Date,p1,p2,p3,p4,Temperature,Rainfall,CPI,Female,Age,Income,p1_m,p2_m,p3_m,p4_m,logit_t,membr_choice
0,1,2023-12-21,10.0,5.0,11.0,7.0,45.34,0.00,100.93,0,60,59662.22,11.938797,6.398548,11.122452,8.794087,1.964666,1
1,1,2023-12-22,14.0,13.0,13.0,10.0,51.87,62.52,100.93,0,60,59662.22,15.354608,7.974386,16.914026,14.223072,-1.842689,0
